In [1]:
import os
import csv
import codecs
import numpy as np
import pandas as pd
np.random.seed(1337)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, merge, LSTM, Lambda, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import sys
from keras.models import Sequential, Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.optimizers import Adagrad

print('Processing text dataset')

Using TensorFlow backend.


Processing text dataset


In [2]:
MAX_SEQUENCE_LENGTH = 3000
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.01

test_data = pd.read_csv('eval1_unlabelled.tsv', delimiter= "\t", encoding='utf-8', header=None)

import pandas as pd
train_file = pd.read_csv('train_file.csv', delimiter= ",", encoding='utf-8', header=None)
train_queries = train_file[0][1:]
train_passages = train_file[1][1:]
train_scores = train_file[2][1:]

test_file = pd.read_csv('test_file.csv', delimiter= ",", encoding='utf-8', header=None)
test_queries = test_file[0][1:]
test_passages = test_file[1][1:]
test_labels = test_data[0]

del test_data

/anaconda/envs/py36tfgpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
texts_1 = train_queries
texts_2 = train_passages
labels = train_scores
print('Found %s texts.' % len(texts_1))

test_texts_1 = test_queries
test_texts_2 = test_passages

print('Found %s texts.' % len(test_texts_1))

del train_queries
del train_passages
del train_scores

print('Tokenizing')

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
texts_1 = np.array(texts_1)
texts_2 = np.array(texts_2)
test_texts_1 = np.array(test_texts_1)
test_texts_2 = np.array(test_texts_2)
all_texts = np.concatenate((texts_1, texts_2, test_texts_1, test_texts_2), axis=None)# + test_texts_1 + test_texts_2
len(texts_1), len(texts_2), len(test_texts_1), len(test_texts_2), len(all_texts)
# all_texts.head()
# tokenizer.fit_on_texts()
tokenizer.fit_on_texts(all_texts)
del all_texts

sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 1524182 texts.
Found 104170 texts.
Tokenizing


/anaconda/envs/py36tfgpu/lib/python3.6/site-packages/keras_preprocessing/text.py:177: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 1004464 unique tokens.


In [6]:
sequences_2

[[372,
  670,
  1147,
  595,
  298,
  187,
  402,
  1088,
  97,
  237,
  787,
  141,
  269,
  4,
  281,
  511,
  787,
  917,
  646,
  7257,
  1145,
  623,
  60,
  787,
  1168,
  127423,
  80,
  395,
  4036,
  1168,
  284],
 [1074,
  106,
  4,
  15419,
  214,
  126,
  761,
  740,
  42,
  15,
  1099,
  286,
  1124,
  3838,
  740,
  42,
  138,
  51,
  15,
  1074,
  106,
  644,
  51,
  761,
  42,
  15,
  1400,
  1303,
  42,
  18797,
  9371,
  303,
  77,
  42,
  18395,
  484,
  15],
 [21,
  2560,
  6319,
  8427,
  1493,
  720,
  7155,
  41,
  1215,
  194051,
  8533,
  21005,
  3,
  610,
  1185,
  7280,
  246,
  5175,
  370,
  530,
  10548,
  2560,
  573,
  4078,
  2560,
  16,
  490,
  49,
  898,
  429,
  182,
  1787,
  74,
  358,
  1696,
  7638,
  898],
 [2800,
  3338,
  44,
  10508,
  370,
  656,
  45,
  17,
  1815,
  3338,
  210,
  571,
  1815,
  1082,
  281,
  10698,
  2360,
  176,
  224,
  3338,
  5668,
  3051,
  1815,
  5633,
  287,
  210,
  3021,
  3252,
  45,
  3848,
  210,
  430,
  

In [4]:
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
# test_labels = np.array(test_labels)
del test_sequences_1
del test_sequences_2
del sequences_1
del sequences_2
del texts_1
del texts_2
del test_texts_1
del test_texts_2

import gc
gc.collect()

print('Preparing embedding matrix.')
# prepare embedding matrix
nb_words = min(MAX_NB_WORDS, len(word_index)) + 1
EMBEDDING_DIM = 300

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

MemoryError: 

In [ ]:
embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
# Model Architecture #
sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = Conv1D(128, 3, activation='relu')(embedded_sequences_1)
x1 = MaxPooling1D(10)(x1)
x1 = Flatten()(x1)
x1 = Dense(64, activation='relu')(x1)
x1 = Dropout(0.2)(x1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = Conv1D(128, 3, activation='relu')(embedded_sequences_2)
y1 = MaxPooling1D(10)(y1)
y1 = Flatten()(y1)
y1 = Dense(64, activation='relu')(y1)
y1 = Dropout(0.2)(y1)

merged = concatenate([x1, y1])#   merge([x1,y1], mode='concat')
merged = BatchNormalization()(merged)
merged = Dense(64, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = BatchNormalization()(merged)
preds = Dense(1, activation='sigmoid')(merged)
model = Model(input=[sequence_1_input,sequence_2_input], output=preds)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
print('model fitting')

model.fit([data_1,data_2], labels, validation_split=VALIDATION_SPLIT,callbacks=callbacks_list, nb_epoch=3, batch_size=1024, shuffle=True)

print('model fitting done')

In [ ]:
model.load_weights("weights.best.hdf5")

print('predicting')

preds = model.predict([test_data_1, test_data_2])
print(preds.shape)

print('predicting done')

In [ ]:
lno=0
outputfile = "answer.tsv"
tempscores=[]  #This will store scores of 10 query,passage pairs as they belong to same query
#f = open(testfile,"r",encoding="utf-8")
fw = open(outputfile,"w",encoding="utf-8")
for i in range(104170):
    #print(row[0])
    Query = test_queries[i]
    Passage = test_passages[i]
    score = preds[i][0]
    tempscores.append(score)
    lno+=1
    if(lno%10==0):
        tempscores = [str(s) for s in tempscores]
        scoreString = "\t".join(tempscores)
        qid = test_labels[i]
        row = str(qid) + "\t" + scoreString + "\n"
        print(row)
        fw.write(row)
        tempscores=[]
    if(lno%5000==0):
        print(lno)
print(lno)
#f.close()
fw.close()
